<a href="https://colab.research.google.com/github/jsedoc/ConceptorDebias/blob/master/EEC_with_CN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip

--2019-02-28 13:48:55--  http://www.saifmohammad.com/WebDocs/EEC/Equity-Evaluation-Corpus.zip
Resolving www.saifmohammad.com (www.saifmohammad.com)... 192.185.17.122
Connecting to www.saifmohammad.com (www.saifmohammad.com)|192.185.17.122|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102568 (100K) [application/zip]
Saving to: ‘Equity-Evaluation-Corpus.zip’

Equity-Evaluation-C 100%[===================>] 100.16K   573KB/s    in 0.2s    

2019-02-28 13:48:55 (573 KB/s) - ‘Equity-Evaluation-Corpus.zip’ saved [102568/102568]



In [2]:
!unzip Equity-Evaluation-Corpus.zip

Archive:  Equity-Evaluation-Corpus.zip
  inflating: Equity-Evaluation-Corpus.csv  
  inflating: README.txt              


In [3]:
import numpy as np
import scipy, requests, codecs, os, re, nltk, itertools, csv
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering, KMeans
import tensorflow as tf
from scipy.stats import spearmanr
import pandas as pd
import functools as ft
import os
import io
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
EEC = pd.read_csv('/content/Equity-Evaluation-Corpus.csv', header=0)

In [0]:
EEC[0:5]


,ID,Sentence,Template,Person,Gender,Race,Emotion,Emotion word
0,2018-En-mystery-05498,Alonzo feels angry.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,angry
1,2018-En-mystery-11722,Alonzo feels furious.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,furious
2,2018-En-mystery-11364,Alonzo feels irritated.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,irritated
3,2018-En-mystery-14320,Alonzo feels enraged.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,enraged
4,2018-En-mystery-14114,Alonzo feels annoyed.,<person subject> feels <emotion word>.,Alonzo,male,African-American,anger,annoyed


# Load GloVe embeddings

In [5]:
!pip install -q gdown
!gdown https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2

Downloading...
From: https://drive.google.com/uc?id=1Ty2exMyi-XOufY-v81RJfiPvnintHuy2
To: /content/gensim_glove.840B.300d.txt.bin
2.65GB [00:47, 56.2MB/s]


# Load Word2vec embeddings
CN performed better using word2vec embeddings. For all 4 sentiments, CN showed improvements on debiasing gender bias.
If using GloVe embeddings, CN showed improvements on anger, fear and joy. For the sentiment 'sadness', CN gave a higher gender difference. However, by my observations, results can be improved if using more female and male names to find conceptors. So with a larger name datasets, the results of 'sadness' may also show improvements on debiasing.

In [6]:
!gdown https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM

Downloading...
From: https://drive.google.com/uc?id=0B7XkCwpI5KDYNlNUTTlSS21pQmM
To: /content/GoogleNews-vectors-negative300.bin.gz
1.65GB [00:37, 44.4MB/s]


In [0]:
!gunzip /content/GoogleNews-vectors-negative300.bin.gz

# Load fasttext

In [8]:
!gdown https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh

Downloading...
From: https://drive.google.com/uc?id=1Zl6a75Ybf8do9uupmrJWKQMnvqqme4fh
To: /content/fasttext.bin
2.42GB [00:36, 66.5MB/s]


In [9]:
import gensim
from gensim.models.keyedvectors import KeyedVectors

fasttext = KeyedVectors.load_word2vec_format('/content/' + 'fasttext.bin', binary=True)
print('The fasttext embedding has been loaded!')

The fasttext embedding has been loaded!


# Load male and female name lists


In [10]:
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
!wget https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt

--2019-02-28 13:59:10--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/male.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20466 (20K) [text/plain]
Saving to: ‘male.txt’

male.txt            100%[===================>]  19.99K  --.-KB/s    in 0.07s   

2019-02-28 13:59:10 (267 KB/s) - ‘male.txt’ saved [20466/20466]

--2019-02-28 13:59:11--  https://www.cs.cmu.edu/Groups/AI/areas/nlp/corpora/names/female.txt
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35751 (35K) [text/plain]
Saving to: ‘female.txt’

female.txt          100%[===================>]  34.91K  --.-KB/s    in 0.07s   

2019-02-28 13:59:11 (472 KB/s) - ‘female.txt’ saved [35751/35751]



In [11]:
!wget https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv

--2019-02-28 13:59:15--  https://gist.githubusercontent.com/mbejda/61eb488cec271086632d/raw/6340b8045b28c2abc0b1d44cfbc80f40284ef890/Black-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 921600 (900K) [text/plain]
Saving to: ‘Black-Male-Names.csv’

Black-Male-Names.cs 100%[===================>] 900.00K  --.-KB/s    in 0.04s   

2019-02-28 13:59:15 (20.3 MB/s) - ‘Black-Male-Names.csv’ saved [921600/921600]



In [12]:
!wget https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv

--2019-02-28 13:59:17--  https://gist.githubusercontent.com/mbejda/6c2293ba3333b7e76269/raw/60aa0c95e8ee9b11b915a26f47480fef5c3203ed/White-Male-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1149042 (1.1M) [text/plain]
Saving to: ‘White-Male-Names.csv’

White-Male-Names.cs 100%[===================>]   1.10M  --.-KB/s    in 0.04s   

2019-02-28 13:59:17 (24.9 MB/s) - ‘White-Male-Names.csv’ saved [1149042/1149042]



In [13]:
!wget https://gist.githubusercontent.com/mbejda/26ad0574eda7fca78573/raw/6936d1a8f5fa5220f2f60a51a06a35b172c50f93/White-Female-Names.csv

--2019-02-28 13:59:20--  https://gist.githubusercontent.com/mbejda/26ad0574eda7fca78573/raw/6936d1a8f5fa5220f2f60a51a06a35b172c50f93/White-Female-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121903 (119K) [text/plain]
Saving to: ‘White-Female-Names.csv’

White-Female-Names. 100%[===================>] 119.05K  --.-KB/s    in 0.02s   

2019-02-28 13:59:20 (5.48 MB/s) - ‘White-Female-Names.csv’ saved [121903/121903]



In [14]:
!wget https://gist.githubusercontent.com/mbejda/9dc89056005a689a6456/raw/bb6ef2375f1289d0ef10dbd8e9469670ac23ceab/Black-Female-Names.csv

--2019-02-28 13:59:22--  https://gist.githubusercontent.com/mbejda/9dc89056005a689a6456/raw/bb6ef2375f1289d0ef10dbd8e9469670ac23ceab/Black-Female-Names.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64757 (63K) [text/plain]
Saving to: ‘Black-Female-Names.csv’

Black-Female-Names. 100%[===================>]  63.24K  --.-KB/s    in 0.01s   

2019-02-28 13:59:22 (4.25 MB/s) - ‘Black-Female-Names.csv’ saved [64757/64757]



In [0]:
glove = KeyedVectors.load_word2vec_format('/content/' + 'gensim_glove.840B.300d.txt.bin', binary=True)

In [0]:
word2vec = KeyedVectors.load_word2vec_format('/content/' + 'GoogleNews-vectors-negative300.bin', binary=True)

# Get frequency list (used in calculating sentence embeddings)

In [17]:
!git clone https://github.com/PrincetonML/SIF

Cloning into 'SIF'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 2.80 MiB | 18.85 MiB/s, done.
Resolving deltas: 100% (56/56), done.


In [0]:
wikiWordsPath = '/content' + '/SIF/auxiliary_data/enwiki_vocab_min200.txt' # https://github.com/PrincetonML/SIF/blob/master/auxiliary_data/enwiki_vocab_min200.txt

frequencies = {}
with open(wikiWordsPath, "r+") as f_in:
    for line in f_in:
        frequencies[line.split(' ')[0]] = float(line.split(' ')[1])

# Load white/black names

In [0]:
word_list = []

In [0]:

f = open('/content/White-Male-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [0]:
len(word_list)

79129

In [0]:
f = open('/content/Black-Male-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [78]:
len(word_list)

79129

In [0]:
f = open('/content/White-Female-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [0]:
f = open('/content/Black-Female-Names.csv', 'r+')
f.readline()
for line in f:
  word_list.append(line.rstrip().split(',')[1].split(' ')[1])
f.close()

In [24]:
len(word_list)

7038

# Load gender names

In [0]:
#load names from larger name dataset
f = open('/content/male.txt', 'r+')
i = 0
word_list= []
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
#load names from larger name dataset
f = open('/content/female.txt', 'r+')
i = 0
for line in f:
  i+=1
  if i < 7:
    continue
  token = line.split()
  word_list.append(token[0])
f.close()

In [0]:
len(word_list)

7944

In [0]:
f = open('/content/name.txt', 'r+')
#word_list = []
for line in f:
  for item in line.rstrip().split(' '):
    word_list.append(item)
f.close()

In [0]:
word_list

# Load Racial Names

In [0]:
# These are names in the paper
word_list = ['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack','Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen']

In [0]:
EEC['Person'].unique()

array(['Alonzo', 'Jamel', 'Alphonse', 'Jerome', 'Leroy', 'Torrance',
       'Darnell', 'Lamar', 'Malik', 'Terrence', 'Adam', 'Harry', 'Josh',
       'Roger', 'Alan', 'Frank', 'Justin', 'Ryan', 'Andrew', 'Jack', 'he',
       'this man', 'this boy', 'my brother', 'my son', 'my husband',
       'my boyfriend', 'my father', 'my uncle', 'my dad', 'Nichelle',
       'Shereen', 'Ebony', 'Latisha', 'Shaniqua', 'Jasmine', 'Tanisha',
       'Tia', 'Lakisha', 'Latoya', 'Amanda', 'Courtney', 'Heather',
       'Melanie', 'Katie', 'Betsy', 'Kristin', 'Nancy', 'Stephanie',
       'Ellen', 'she', 'this woman', 'this girl', 'my sister',
       'my daughter', 'my wife', 'my girlfriend', 'my mother', 'my aunt',
       'my mom', 'him', 'her'], dtype=object)

In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
#For word2vec data
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
cn_embeddings.shape

(7099, 300)

In [0]:
#Sentence embeddings

In [0]:
import nltk

class Sentence:
    
    def __init__(self, sentence):
        self.raw = sentence
        normalized_sentence = sentence.replace("‘", "'").replace("’", "'")
        self.tokens = [t.lower() for t in nltk.word_tokenize(normalized_sentence)]

In [0]:
#Raw Sentence Embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

def run_sif_benchmark(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
#    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    #average sentiment embedding based on several emotional words of the same sentiment
#    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis=0)    
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])


    return proj1, proj2

In [0]:
#CN Sentence Embeddings
def run_conceptor_benchmark(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha =2): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
 #   sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
 #   sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis = 0) 
 #   print(sen_embeddings)
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
        
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    #negC = np.eye(300) - SWC[model_str]
    
    
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    proj1 = embedding1.dot(model[sentiment])/norm(model[sentiment])
    proj2 = embedding2.dot(model[sentiment])/norm(model[sentiment])
    return proj1, proj2

In [0]:
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])
#  p_raw.append([p1.statistic, p2.pvalue])
#  print('race difference on Raw and CN data:')
#  print(abs((sum(aa)-sum(ea))/len(aa)), abs((sum(aa_cn)-sum(ea_cn))/len(aa_cn)))
#  print('t value and p value :')
#  print(p1,p2)

In [83]:

print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
#p['sentiment'] = ['anger','fear','joy','sadness']
#p.reset_index('sentiment')
print(p)


RAW vs CN data using WORD2VEC, male
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          19.359635  3.104938e-57       16.952476  1.884045e-47
fear           21.366772  2.309479e-65       20.151225  1.896418e-60
joy            11.707836  6.245311e-27       10.351089  4.471070e-22
sadness        17.006111  1.142249e-47       16.881343  3.657869e-47


In [35]:
p1.pvalue, p1.statistic

(1.1422485946061406e-47, 17.006110741058613)

In [0]:
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [56]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          13.849456  4.572745e-35        5.564122  5.255272e-08
fear            8.261900  3.016258e-15       -2.214267  2.745619e-02
joy             5.400960  1.228679e-07        1.979456  4.855011e-02
sadness        19.543729  5.545095e-58       15.293601  9.001127e-41


In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p =[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='male')]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [86]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -39.229215  5.467233e-130      -28.314121   1.967884e-92
fear          -53.682502  1.007069e-170      -45.070483  1.457789e-147
joy           -36.353356  1.032966e-120      -33.692194  9.716276e-112
sadness       -22.533354   4.817098e-70      -26.089576   5.333202e-84


In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='female')]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [59]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic    raw_p_value  cn_t_statistic    cn_p_value
anger         -25.594525   4.332972e-82      -18.320194  5.245264e-53
fear          -33.008122  2.269988e-109      -25.057505  5.264661e-80
joy           -29.708214   1.378851e-97      -21.810645  3.782584e-67
sadness       -15.018152   1.128162e-39      -14.605255  4.903266e-38


In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='male')]
  #print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [89]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -24.943738   1.461045e-79      -18.225032   1.278958e-52
fear          -38.831695  9.875491e-129      -33.469088  5.717150e-111
joy           -23.820355   3.732282e-75      -22.105511   2.481160e-68
sadness       -19.571735   4.267023e-58      -15.474926   1.695819e-41


In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') & (sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European') & (sent_sent['Gender']=='female')]
  #print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [62]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger           9.036385  1.130757e-17        3.902433  1.142794e-04
fear            3.073295  2.283697e-03       -4.647903  4.761504e-06
joy           -16.240469  1.423592e-44       -7.902355  3.615591e-14
sadness         4.786356  2.512300e-06       -0.268713  7.883090e-01


# Using averaged sentiment embeddings

In [0]:
#Raw Sentence Embeddings
from sklearn.decomposition import TruncatedSVD, randomized_svd
from numpy.linalg import norm

def run_sif_benchmark(sentiment, sent1, sent2, model_str, freqs={}, a=0.001): 
    total_freq = sum(freqs.values())
    model = eval(model_str)
    embeddings = []
    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    #average sentiment embedding based on several emotional words of the same sentiment
    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis=0)    
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.array(np.average([model[token] for token in tokens1], axis=0, weights=weights1))
    embedding1 /= norm(embedding1)
    embedding2 = np.array(np.average([model[token] for token in tokens2], axis=0, weights=weights2))
    embedding2/= norm(embedding2)
    proj1 = embedding1.dot(sen_embeddings)/norm(sen_embeddings)
    proj2 = embedding2.dot(sen_embeddings)/norm(sen_embeddings)


    return proj1, proj2

In [0]:
#CN Sentence Embeddings
def run_conceptor_benchmark(sentiment,sent1, sent2, model_str, freqs={}, a=0.001, alpha =2): 
    total_freq = sum(freqs.values())
    
    model = eval(model_str)
    embeddings = []
    sen_list = EEC[EEC['Emotion']==sen]['Emotion word'].unique()
    sen_embeddings = np.average([model[item] for item in sen_list if item in model.vocab], axis = 0) 
 #   print(sen_embeddings)
    # SIF requires us to first collect all sentence embeddings and then perform 
    # common component analysis.
        
    tokens1 =  sent1.tokens
    tokens2 =  sent2.tokens
        
    tokens1 = [token for token in tokens1 if token in model and token.islower()]
    tokens2 = [token for token in tokens2 if token in model and token.islower()]
        
    weights1 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens1]
    weights2 = [a/(a+freqs.get(token,0)/total_freq) for token in tokens2]
        
    embedding1 = np.average([model[token] for token in tokens1], axis=0, weights=weights1)
    embedding2 = np.average([model[token] for token in tokens2], axis=0, weights=weights2)
        

    R = (cn_embeddings.T).dot(cn_embeddings) / cn_embeddings.shape[0]

    
    
    C = R @ np.linalg.inv (R + alpha ** (-2) * np.eye(300))# calculate the conceptor matrix
    
    negC = np.eye(300) - C
    #negC = np.eye(300) - SWC[model_str]
    
    
    embedding1 = negC.dot(embedding1.T).T
    embedding2 = negC.dot(embedding2.T).T
    embedding1 /= norm(embedding1)
    embedding2 /= norm(embedding2)
    proj1 = embedding1.dot(sen_embeddings)/norm(sen_embeddings)
    proj2 = embedding2.dot(sen_embeddings)/norm(sen_embeddings)
    return proj1, proj2

In [0]:
cn_embeddings = []
cn_embeddings.append([word2vec[word] for word in word_list if word in word2vec.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [94]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic   raw_p_value  cn_t_statistic    cn_p_value
anger          18.876432  2.863949e-55       10.708958  2.497658e-23
fear           18.281446  7.540234e-53        9.122867  5.946521e-18
joy            14.240858  1.340659e-36        7.094972  7.256353e-12
sadness        15.365000  4.667011e-41        9.472993  4.250451e-19


In [0]:
p =[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'word2vec', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [67]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic   raw_p_value  cn_t_statistic  cn_p_value
anger           6.118688  2.533850e-09        0.521135    0.602603
fear            4.115710  4.820979e-05       -1.813532    0.070609
joy             1.940412  5.313473e-02       -0.211603    0.832540
sadness         2.070742  3.911770e-02       -1.184341    0.237084


In [0]:
#For GloVe data
cn_embeddings = []
cn_embeddings.append([glove[word] for word in word_list if word in glove.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p = []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [97]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -33.357260  1.393124e-110      -39.290941  3.494110e-130
fear          -41.600778  2.538149e-137      -43.037422  1.246651e-141
joy           -48.753785  7.681814e-158      -40.894821  3.616948e-135
sadness       -40.561857  3.825451e-134      -40.225661  4.193128e-133


In [0]:
p= []
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'glove', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [70]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic    raw_p_value  cn_t_statistic    cn_p_value
anger         -24.210266   1.087004e-76      -24.209658  1.092996e-76
fear          -33.929640  1.483594e-112      -29.540503  5.677200e-97
joy           -30.262349   1.316721e-99      -27.768400  2.185446e-90
sadness       -33.264375  2.922746e-110      -26.979861  2.099917e-87


In [0]:
#For Fasttext data
cn_embeddings = []
cn_embeddings.append([fasttext[word] for word in word_list if word in fasttext.vocab])
cn_embeddings = np.array(cn_embeddings[0])

In [0]:
p =[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='male')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='male') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])



In [100]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic    raw_p_value  cn_t_statistic     cn_p_value
anger         -46.225740  7.524439e-151      -42.788945  6.822388e-141
fear          -40.925205  2.918379e-135      -37.277705  9.673073e-124
joy           -43.957295  2.444131e-144      -36.843588  2.525190e-122
sadness       -44.227992  3.970227e-145      -34.961843  4.557815e-116


In [0]:
p=[]
for sen in ['anger','fear','joy', 'sadness']:
  aa = []
  ea = []
  aa_cn = []
  ea_cn = []
  sent_sent = []
  sent_aa = []
  sent_ea = []
  sent_sent = EEC[EEC['Emotion']==sen]
  sent_aa = sent_sent[(sent_sent['Race']=='African-American') &(sent_sent['Gender']=='female')]
  sent_ea = sent_sent[(sent_sent['Race']=='European')&(sent_sent['Gender']=='female') ]
 # print(len(sent_ea))
  for sen1, sen2 in zip(sent_aa['Sentence'], sent_ea['Sentence']):
    sentence1=Sentence(sen1)
    sentence2=Sentence(sen2)
    score1, score2 = run_sif_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    score3, score4 = run_conceptor_benchmark(sen, sentence1, sentence2, 'fasttext', frequencies, 0.0001)
    aa.append(score1)
    ea.append(score2)
    aa_cn.append(score3)
    ea_cn.append(score4)
  p1 = scipy.stats.ttest_rel(aa, ea)
  #difference is significant, we can reject the null hypothesis that they are identical
  p2 = scipy.stats.ttest_rel(aa_cn, ea_cn)
  p.append([p1.statistic, p1.pvalue, p2.statistic, p2.pvalue])


In [74]:
print('RAW vs CN data using WORD2VEC, male')
p = pd.DataFrame(p, index=['anger','fear','joy','sadness'], columns= ['raw_t_statistic', 'raw_p_value', 'cn_t_statistic', 'cn_p_value'])
print(p)

RAW vs CN data using WORD2VEC, male
         raw_t_statistic   raw_p_value  cn_t_statistic  cn_p_value
anger          12.255394  5.795978e-29        1.919071    0.055791
fear            7.172205  4.445886e-12       -3.024353    0.002676
joy            -0.056098  9.552957e-01       -3.667741    0.000283
sadness         5.408571  1.181476e-07       -3.532512    0.000467
